# Examen ETL: SPARK 09/02

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de DataSetPartidos.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen o en su defecto por email.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: ¿ En que se diferencian las acciones de las transformaciones en SPARK? ¿Podrías ponerme un par de ejemplo de cada una de ellas y describir que hacen, para que sirve?

* __Transformaciones:__
Las transformaciones definirán como cambiará el flujo de información generando un nuevo RDD. Con estas transformaciones no se está evaluando el RDD, sino creando uno nuevo. Algunas de las transformaciones son:

  * map: aplica una función a cada elemento de la colección:
  * filter: selecciona el subconjunto de elementos que cumplen una determinada expresión booleana:
  * flatMap: además de realizar una función map, aplica un método flatten:
 
* __Acciones:__
Las acciones nos permitirán evaluar un RDD y devolver un resultado. De esta forma se ejecuta todo el flujo de datos definido. ¿Algún ejemplo? Aquí van algunos:

  * count: nos devuelve el número total de elemento.
  * collect: nos vuelca toda la colección distribuida en un array en memoria.
Ojo, cuidao. Si el RDD es muy grande, podemos tener problemas al volcar toda la colección en memoria.

## Segunda tarea: Inicializar spark context y cargar los datos desde el fichero.

In [1]:
from pyspark import SparkContext
sc = SparkContext('local', 'simple App')

In [7]:
partidos = "../DataSetPartidos.txt"
raw_data = sc.textFile(partidos)

In [19]:
from pyspark.sql import Row

csv_data = raw_data.map(lambda x: x.split("::"))
row_data = csv_data.map(lambda p: Row(
    obs=int(p[0]), 
    temporada=p[1],
    jornada=p[2],
    equipo_local=p[3],
    equipo_vist=p[4],
    gol_local=int(p[5]),
    gol_vist=int(p[6])
    )
)

In [11]:
print(row_data)

PythonRDD[6] at RDD at PythonRDD.scala:48


In [20]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [221]:
partidos = sqlContext.createDataFrame(row_data)
partidos.registerTempTable("partidos")

In [22]:
partidos.show()

+------------------+------------------+---------+--------+-------+---+---------+
|      equipo_local|       equipo_vist|gol_local|gol_vist|jornada|obs|temporada|
+------------------+------------------+---------+--------+-------+---+---------+
|     Athletic Club|         Barcelona|        1|       1|      1|  1|  1970-71|
|        Las Palmas|Atletico de Madrid|        1|       1|      1|  2|  1970-71|
|       Real Madrid|          Valencia|        2|       0|      1|  3|  1970-71|
|     Celta de Vigo| Sporting de Gijon|        2|       0|      1|  4|  1970-71|
|             Elche|           Granada|        1|       1|      1|  5|  1970-71|
|          Espanyol|           Sevilla|        0|       1|      1|  6|  1970-71|
|          Sabadell|     Real Sociedad|        0|       0|      1|  7|  1970-71|
|     Real Zaragoza|         CD Malaga|        0|       0|      1|  8|  1970-71|
|          Valencia|        Las Palmas|        5|       1|      2|  9|  1970-71|
|Atletico de Madrid|     Ath

## Tercera pregunta: Calcular el número total de goles que ha marcado el Real Sporting de Gijón.

* __CON SQL__

In [24]:
query = "SELECT SUM(gol_local) FROM partidos WHERE equipo_local='Sporting de Gijon'"
golesSportingLocal = sqlContext.sql(query)

In [16]:
golesSportingLocal.show()

+---------------+
|sum(goleslocal)|
+---------------+
|           1275|
+---------------+



In [25]:
query1 = "SELECT SUM(gol_vist) FROM partidos WHERE equipo_vist='Sporting de Gijon'"
golesSportingVis = sqlContext.sql(query1)

In [26]:
goles_Sporting_Vis.show()

+-------------+
|sum(gol_vist)|
+-------------+
|          767|
+-------------+



In [27]:
goles

2042

* __CON SPARK SQL__

In [30]:
filtroLocal = partidos.equipo_local == 'Sporting de Gijon'

In [29]:
filtroVis = partidos.equipo_vist == 'Sporting de Gijon'

In [32]:
partidos.filter(filtroLocal).groupBy().sum("gol_local").show()

+--------------+
|sum(gol_local)|
+--------------+
|          1275|
+--------------+



In [33]:
partidos.filter(filtroVis).groupBy().sum("gol_vist").show()

+-------------+
|sum(gol_vist)|
+-------------+
|          767|
+-------------+



In [35]:
1275 + 767

2042

## Cuarta pregunta: ¿ En que temporada se marcaron más goles?

* __ CON SQL__

In [95]:
query3 = "SELECT temporada, sum(gol_vist + gol_local) as goles_temporada FROM partidos group by temporada order by -goles_temporada "
goles_temporada = sqlContext.sql(query3)

In [96]:
goles_temporada.show(1)

+---------+---------------+
|temporada|goles_temporada|
+---------+---------------+
|  2012-13|           2294|
+---------+---------------+
only showing top 1 row



* __ CON SPARK SQL __

* En primer lugar vamos a importar "desc" de pyspark.sql.functions para que:

In [36]:
from pyspark.sql.functions import desc

* Creamos una nueva columna en nuestro dataset que llamaremos 'goles_partido' y esta será una suma de los goles locales más los visitantes.

In [38]:
partidos = partidos.\
    withColumn('goles_partido', partidos.gol_local + partidos.gol_vist)

* Por último, a partir de un __groupBy__ agrupamos por temporadas sumando la columna creada anteriormente ('goles_partido') y, utilizando el comando .sort() establecemos el "desc" anteriormente importado que ordena de mayor a menor los resultados.

* Cuando lo tengamos lo único que tendremos que hacer es un .show() del primer dato.

In [45]:
golesTemp = partidos.\
    select("temporada","goles_partido").\
    groupBy('temporada').\
    sum('goles_partido').\
    persist()
    
golesTemp.sort(desc('sum(goles_partido)')).show(1)

+---------+------------------+
|temporada|sum(goles_partido)|
+---------+------------------+
|  2012-13|              2294|
+---------+------------------+
only showing top 1 row



Fue en la temporada __2012-2013__ en la que se marcaron el mayor número de goles (__2294 goles__) comparado con el resto de temporadas.

## Quinta pregunta: ¿Cúal es el equipo que tiene el record de más goles como local? ¿ y cómo visitante?

* __ CON SQL__

In [102]:
query5 = "SELECT equipo_local, sum(gol_local) as goles_totales FROM partidos group by equipo_local order by -goles_totales "
goles_local = sqlContext.sql(query5)

In [104]:
goles_local.show(1)

+------------+-------------+
|equipo_local|goles_totales|
+------------+-------------+
| Real Madrid|         2054|
+------------+-------------+
only showing top 1 row



In [97]:
query4 = "SELECT equipo_vist, sum(gol_vist) as goles_totales FROM partidos group by equipo_vist order by -goles_totales "
goles_visit = sqlContext.sql(query4)

In [99]:
goles_visit.show(2)

+-----------+-------------+
|equipo_vist|goles_totales|
+-----------+-------------+
|Real Madrid|         1296|
|  Barcelona|         1296|
+-----------+-------------+
only showing top 2 rows



* __ CON SPARK SQL__

* __MÁS GOLES COMO LOCAL:__

In [44]:
local = partidos.\
    groupBy('equipo_local').\
    sum('gol_local').\
    sort(desc('sum(gol_local)'))
    
local.show(1)

+------------+--------------+
|equipo_local|sum(gol_local)|
+------------+--------------+
| Real Madrid|          2054|
+------------+--------------+
only showing top 1 row



Podemos observar en los resultados obtenidos, que el equipo que tiene el récord de más goles como local es el __Real Madrid__ con un total de __2054__ goles a favor.

* __MÁS GOLES COMO VISITANTE:__

In [46]:
Visitante = partidos.\
    groupBy('equipo_vist').\
    sum('gol_vist').\
    sort(desc('sum(gol_vist)'))
    
Visitante.show(2)

+-----------+-------------+
|equipo_vist|sum(gol_vist)|
+-----------+-------------+
|Real Madrid|         1296|
|  Barcelona|         1296|
+-----------+-------------+
only showing top 2 rows



Con los resultados obtenidos, podemos decir que el equipo que ha conseguido el récord de más goles como visitante ha sido también el __Real Madrid__ con un total de __1296__ goles, empatado con el __Barsa__

## Sexta pregunta: ¿Cúales son las 3 décadas en las que más goles se metieron?

* __CON SQL__

In [120]:
query6 = "SELECT concat(substring(temporada,3,1),'0s') as temporada, sum(gol_vist + gol_local) as goles_decada " + \
         "FROM partidos " + \
         "group by substring(temporada, 3, 1) " +\
         "order by -goles_decada "
decada_goles_partido = sqlContext.sql(query6)

In [121]:
decada_goles_partido.show(3)

+---------+------------+
|temporada|goles_decada|
+---------+------------+
|      00s|       20759|
|      90s|       19426|
|      80s|       17351|
+---------+------------+
only showing top 3 rows



* __CON SPARK SQL__

In [48]:
import pyspark.sql.functions as psf

In [139]:
substring_col = psf.concat(psf.substring(partidos['temporada'],3,1), psf.lit('0s'))

In [140]:
df1 = partidos.withColumn('decada', substring_col).\
    groupBy('decada').\
    sum('goles_partido').\
    sort(desc('sum(goles_partido)'))
    
df1.show(3)

+------+------------------+
|decada|sum(goles_partido)|
+------+------------------+
|   00s|             20759|
|   90s|             19426|
|   80s|             17351|
+------+------------------+
only showing top 3 rows



##  Séptima pregunta: ¿Cúal es la media de victorias de los equipos que han estado menos de 10 temporadas en primera división?

* __CON SQL__

Vamos a calcular todos los equipos con menos de 10 temporadas activos
Miramos visitante o local, con eso vale

In [223]:
partidos.show()

+------------------+------------------+---------+--------+-------+---+---------+
|      equipo_local|       equipo_vist|gol_local|gol_vist|jornada|obs|temporada|
+------------------+------------------+---------+--------+-------+---+---------+
|     Athletic Club|         Barcelona|        1|       1|      1|  1|  1970-71|
|        Las Palmas|Atletico de Madrid|        1|       1|      1|  2|  1970-71|
|       Real Madrid|          Valencia|        2|       0|      1|  3|  1970-71|
|     Celta de Vigo| Sporting de Gijon|        2|       0|      1|  4|  1970-71|
|             Elche|           Granada|        1|       1|      1|  5|  1970-71|
|          Espanyol|           Sevilla|        0|       1|      1|  6|  1970-71|
|          Sabadell|     Real Sociedad|        0|       0|      1|  7|  1970-71|
|     Real Zaragoza|         CD Malaga|        0|       0|      1|  8|  1970-71|
|          Valencia|        Las Palmas|        5|       1|      2|  9|  1970-71|
|Atletico de Madrid|     Ath

In [227]:
query7 = "SELECT equipo_vist, count(distinct(temporada)) as total_temporadas " + \
    "FROM partidos " + \
    "group by equipo_vist " + \
    "having count(distinct(temporada)) <  10 "
equipos_menos10 = sqlContext.sql(query7)

In [228]:
equipos_menos10.show(2)

+-----------+----------------+
|equipo_vist|total_temporadas|
+-----------+----------------+
|  Barakaldo|               6|
|   Malaga B|               3|
+-----------+----------------+
only showing top 2 rows



In [229]:
query8 = "SELECT CASE " +\
                    "WHEN (gol_local > gol_vist) THEN equipo_local "+ \
                    "WHEN (gol_local < gol_vist) THEN equipo_vist " + \
                "END AS ganador, temporada "+\
                "FROM partidos "

In [230]:
ganadores_temporada = sqlContext.sql(query8)

In [324]:
ganadores_temporada.groupBy("ganador").agg(psf.count(psf.col("temporada"))).show(10)

+-------------------+----------------+
|            ganador|count(temporada)|
+-------------------+----------------+
|           Logroñes|             233|
|           Espanyol|             584|
|            Badajoz|             140|
|Racing de Santander|             541|
|          Barakaldo|              78|
|           Malaga B|              31|
|           Mirandes|              38|
|         Real Union|              12|
|  Sporting de Gijon|             601|
|          Ontinyent|               8|
+-------------------+----------------+
only showing top 10 rows



Una vez obtenidos todos los ganadores de los partidos y la temporada del mismo
Vamos registrar la tabla para poder filtrar mediante join con la anterior

In [231]:
ganadores_temporada.registerTempTable("ganadores")

In [232]:
equipos_menos10.registerTempTable("equipos_menos10")

In [292]:
query9 = "SELECT ganador, count(temporada) as victorias, "+\
         " count(distinct(temporada)) as total_temporada, count(temporada)/count(distinct(temporada)) as media_victorias "+\
         "FROM ganadores AS g "+\
         "JOIN equipos_menos10 AS e "+\
         "ON g.ganador == e.equipo_vist "+\
         "GROUP BY ganador"
final_menos10 = sqlContext.sql(query9)

Si pensamos que es la media para cada equipo ya estaríamos

In [294]:
final_menos10.show(2)

+---------+---------+---------------+------------------+
|  ganador|victorias|total_temporada|   media_victorias|
+---------+---------+---------------+------------------+
|Barakaldo|       78|              6|              13.0|
| Malaga B|       31|              3|10.333333333333334|
+---------+---------+---------------+------------------+
only showing top 2 rows



Si penamos que es la media total

In [ ]:
final_menos10.registerTempTable("final_menos10")

In [260]:
query10 = "SELECT sum(victorias)/sum(total_temporadas) FROM final_menos10"
sqlContext.sql(query10).show()

+------------------------------------------------------------------------+
|(CAST(sum(victorias) AS DOUBLE) / CAST(sum(total_temporadas) AS DOUBLE))|
+------------------------------------------------------------------------+
|                                                      12.076923076923077|
+------------------------------------------------------------------------+



* __CON SPARK SQL__

In [303]:
def ganador(gol_local,gol_vist,equipo_local,equipo_vist):
    if (gol_local > gol_vist):
        ganador = equipo_local
    elif (gol_local < gol_vist):
        ganador = equipo_vist
    else:
        ganador = ""
    return ganador    

Las udfs son las funciones típicas de spark SQL, si queremos aplicar una funcion de python dentro de un DataFrame como haríamos 
con un map dentro de un rdd tenmos que usar estas udfs que se definen como se ve a continuación, poniendo el nombre de la función 
y su formato de salida

In [304]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

ganador_udf = udf(ganador, StringType())

In [308]:
ganadores = partidos. \
    withColumn('ganador', ganador_udf(psf.col('gol_local'), 
                                                     psf.col("gol_vist"), 
                                                     psf.col('equipo_local'), 
                                                     psf.col('equipo_vist'))). \
    select('equipo_vist','temporada'). \
    groupBy('equipo_vist').\
    agg(psf.count("temporada").alias("total_victorias"),
        psf.countDistinct("temporada").alias("total_temporadas")).\
    withColumn("media", psf.col("total_victorias")/psf.col("total_temporadas")).\
    where(psf.col("total_temporadas") < 10)

In [310]:
ganadores.show(20)

+----------------+---------------+----------------+------------------+
|     equipo_vist|total_victorias|total_temporadas|             media|
+----------------+---------------+----------------+------------------+
|       Barakaldo|            114|               6|              19.0|
|        Malaga B|             63|               3|              21.0|
|        Mirandes|             59|               3|19.666666666666668|
|      Real Union|             21|               1|              21.0|
| Lorca Deportiva|             42|               2|              21.0|
|       Ontinyent|             19|               1|              19.0|
|       Algeciras|             78|               4|              19.5|
|      AD Almeria|             72|               4|              18.0|
|      Mallorca B|             21|               1|              21.0|
|    FC Cartagena|             63|               3|              21.0|
|Getafe Deportivo|            114|               6|              19.0|
|     

In [318]:
ganadores.select("total_victorias", "total_temporadas").groupBy().sum().show()

+--------------------+---------------------+
|sum(total_victorias)|sum(total_temporadas)|
+--------------------+---------------------+
|                4361|                  221|
+--------------------+---------------------+



##  Octava pregunta: ¿Cúal es la media de goles como visitante por partido?

* __MEDIA DE GOLES COMO LOCAL POR PARTIDO__

__ CON SQL__

En primer lugar vamos a definir nuestra query para posteriormente ejecutarla con SQL, lo que le estamos diciendo es que nos seleccione de el promedio de la columna "goleslocal" del dataset de partidos.

En segundo lugar ejecutamos con sqlContext.sql() y hacemos un .show() para visualizar el resultado.

In [141]:
query = "SELECT AVG(gol_local) FROM partidos"
mediaGolesLoc = sqlContext.sql(query)
mediaGolesLoc.show()

+-----------------+
|   avg(gol_local)|
+-----------------+
|1.519004577321977|
+-----------------+



 __ CON SPARK SQL__

Para realizarlo con Spark vamos a realizar un groupBy() del dataset de partidos seleccionando la media del total de la columna "goleslocal" para determinar los goles de media que se marcan por partido como local.

In [144]:
MGolesLoc = partidos.\
    groupBy().\
    mean("gol_local")
    
MGolesLoc.show()

+-----------------+
|   avg(gol_local)|
+-----------------+
|1.519004577321977|
+-----------------+



* __MEDIA DE GOLES COMO VISITANTE POR PARTIDO__
    * Llevamos a cabo el mismo proceso para calcular los goles de media como visitante.

__CON SQL__

In [145]:
query = "SELECT AVG(gol_vist) FROM partidos"
mediaGolesVis = sqlContext.sql(query)
mediaGolesVis.show()

+------------------+
|     avg(gol_vist)|
+------------------+
|0.9280085752361087|
+------------------+



 __CON SPARK__

In [147]:
MGolesVis = partidos.\
    groupBy().\
    mean("gol_vist")
    
MGolesVis.show()

+------------------+
|     avg(gol_vist)|
+------------------+
|0.9280085752361087|
+------------------+

